In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [2]:
# Normalization values for CIFAR-10
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Bottleneck(nn.Module):
    expansion = 4  # Multiplicative factor for output channels in the 3rd conv

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()

        # 1x1 conv to reduce channels
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1   = nn.BatchNorm2d(planes)

        # 3x3 conv
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(planes)

        # 1x1 conv to expand channels back by expansion
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3   = nn.BatchNorm2d(planes * self.expansion)

       
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out, inplace=True)
        return out


In [4]:
class ResNet164(nn.Module):
    def __init__(self, block=Bottleneck, num_blocks=[18, 18, 18], num_classes=10):
        super(ResNet164, self).__init__()
        self.in_planes = 16

        # Initial 3x3 convolution
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(16)

        
        self.layer1 = self._make_layer(block, planes=16,  num_blocks=num_blocks[0], stride=1)
        
        self.layer2 = self._make_layer(block, planes=32,  num_blocks=num_blocks[1], stride=2)
        
        self.layer3 = self._make_layer(block, planes=64,  num_blocks=num_blocks[2], stride=2)
        
        self.linear = nn.Linear(64 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

       
        out = F.avg_pool2d(out, out.size(3))  # or out.shape[2]
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [5]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, testloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Accumulate loss
            running_loss += loss.item() * images.size(0)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / total
    test_acc = 100.0 * correct / total

    return test_loss, test_acc

In [6]:
model = ResNet164().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=0.1, momentum=0.9, weight_decay=1e-4
)


In [7]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [8]:
reject_diffs = []

# Evaluate reject differential over the test set
for i, (images, labels) in enumerate(tqdm(testloader)):
    
    images = images.to(device)

    # Original prediction
    with torch.no_grad():
        original_output = model(images)
        original_prob = F.softmax(original_output, dim=1)               # shape [batch_size, 10]
        original_confidence, original_pred = torch.max(original_prob, 1) # shape [batch_size]

    # For metamorphic transform
    for tf in transformations:
       
        transformed_images = tf(images.cpu()).to(device)

        with torch.no_grad():
            transformed_output = model(transformed_images)
            transformed_prob = F.softmax(transformed_output, dim=1)
            transformed_confidence, transformed_pred = torch.max(transformed_prob, 1)

        # Compute absolute difference in confidence for each sample in the batch
    
        diff_tensor = torch.abs(original_confidence - transformed_confidence)

        # Convert to a single scalar by taking the mean over the batch
        diff_mean = diff_tensor.mean().item()

       
        reject_diffs.append(diff_mean)

       
        print(f"Batch {i} | Transform: {tf.__class__.__name__} | "
              f"Reject Differential (mean): {diff_mean:.4f}")


print("Number of recorded reject differentials:", len(reject_diffs))
print("Example of first few reject_diffs:", reject_diffs[:5])


  1%|█▋                                                                                                                                                                   | 1/100 [00:00<01:22,  1.20it/s]

Batch 0 | Transform: RandomRotation | Reject Differential (mean): 0.0353
Batch 0 | Transform: ColorJitter | Reject Differential (mean): 0.0743
Batch 0 | Transform: RandomAffine | Reject Differential (mean): 0.0361
Batch 1 | Transform: RandomRotation | Reject Differential (mean): 0.0000


  2%|███▎                                                                                                                                                                 | 2/100 [00:01<00:44,  2.21it/s]

Batch 1 | Transform: ColorJitter | Reject Differential (mean): 0.0701
Batch 1 | Transform: RandomAffine | Reject Differential (mean): 0.0484
Batch 2 | Transform: RandomRotation | Reject Differential (mean): 0.0376
Batch 2 | Transform: ColorJitter | Reject Differential (mean): 0.0928


  4%|██████▌                                                                                                                                                              | 4/100 [00:01<00:25,  3.81it/s]

...

Batch 97 | Transform: ColorJitter | Reject Differential (mean): 0.0774
Batch 97 | Transform: RandomAffine | Reject Differential (mean): 0.0453
Batch 98 | Transform: RandomRotation | Reject Differential (mean): 0.0395
Batch 98 | Transform: ColorJitter | Reject Differential (mean): 0.0959


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  6.03it/s]

Batch 98 | Transform: RandomAffine | Reject Differential (mean): 0.0419
Batch 99 | Transform: RandomRotation | Reject Differential (mean): 0.0397
Batch 99 | Transform: ColorJitter | Reject Differential (mean): 0.1217
Batch 99 | Transform: RandomAffine | Reject Differential (mean): 0.0490


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.75it/s]

Number of recorded reject differentials: 300
Example of first few reject_diffs: [0.03526679426431656, 0.07425025850534439, 0.036133166402578354, 0.0, 0.07010944187641144]


In [9]:

num_epochs = 20  
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, trainloader, criterion, optimizer)
    val_loss, val_acc     = evaluate(model, testloader, criterion)

    print(f"[Epoch {epoch+1}/{num_epochs}] "f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "f"Test Loss: {val_loss:.4f},  Test Acc: {val_acc:.2f}%")


[Epoch 1/20] Train Loss: 2.4945, Train Acc: 10.00% | Test Loss: 2.3040,  Test Acc: 10.00%
[Epoch 2/20] Train Loss: 2.2999, Train Acc: 10.42% | Test Loss: 2.2674,  Test Acc: 13.13%
[Epoch 3/20] Train Loss: 2.0387, Train Acc: 20.93% | Test Loss: 1.9176,  Test Acc: 27.33%
[Epoch 4/20] Train Loss: 1.8531, Train Acc: 28.21% | Test Loss: 1.8314,  Test Acc: 28.91%
[Epoch 5/20] Train Loss: 1.7456, Train Acc: 33.49% | Test Loss: 1.7104,  Test Acc: 36.44%
[Epoch 6/20] Train Loss: 1.5997, Train Acc: 40.63% | Test Loss: 1.6092,  Test Acc: 40.95%
[Epoch 7/20] Train Loss: 1.4489, Train Acc: 47.18% | Test Loss: 1.3487,  Test Acc: 50.94%
[Epoch 8/20] Train Loss: 1.3036, Train Acc: 52.75% | Test Loss: 1.2456,  Test Acc: 55.68%
[Epoch 9/20] Train Loss: 1.1702, Train Acc: 58.08% | Test Loss: 1.2348,  Test Acc: 56.13%
[Epoch 10/20] Train Loss: 1.0686, Train Acc: 61.71% | Test Loss: 1.0648,  Test Acc: 62.06%
[Epoch 11/20] Train Loss: 0.9904, Train Acc: 64.56% | Test Loss: 1.0648,  Test Acc: 63.73%
[Epoch 1

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score

def validate(model, testloader, device, criterion):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    
    # Switch off gradient tracking for validation
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            
            # Forward pass
            output = model(data)
            loss = criterion(output, target)
            
            # Accumulate loss * batch_size for accurate average
            val_running_loss += loss.item() * data.size(0)
            
            # Predictions
            _, preds = torch.max(output, 1)
            val_running_correct += (preds == target).sum().item()
            
            # Store for metrics
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    # Calculate average loss
    total_samples = len(testloader.dataset)
    val_loss = val_running_loss / total_samples
    
    # Calculate accuracy
    val_accuracy = 100.0 * val_running_correct / total_samples
    
    # Calculate precision, recall, F1 (weighted or macro—your choice)
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall    = recall_score(all_targets, all_preds, average='weighted')
    f1        = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1


In [11]:
model_save_path = 'resnet164_RejDiff_cifar10.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to resnet164_RejDiff_cifar10.pth


In [12]:
model.load_state_dict(torch.load(model_save_path))
mode = model.to(device)

/tmp/ipykernel_330611/1717748095.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


In [13]:
final_val_loss, final_val_accuracy, final_precision, final_recall, final_f1 = validate( model,  testloader, device=device, criterion=criterion)

print(f"\nFinal Test Results - Loss: {final_val_loss:.4f}, "f"Accuracy: {final_val_accuracy:.2f}%, "f"Precision: {final_precision:.2f}, "f"Recall: {final_recall:.2f}, "f"F1 Score: {final_f1:.2f}")


Final Test Results - Loss: 0.6977, Accuracy: 76.82%, Precision: 0.79, Recall: 0.77, F1 Score: 0.76
